## Standardized test cases

For benchmarking, we need a standardized set of case studies, ranging form simple to very large. This helps us to test and track the performance of improvements of bw_timex with the same set of test cases.


### 0) set up variables & functions

In [20]:
import numpy as np
import bw2data as bd

from bw_temporalis import TemporalDistribution
from datetime import datetime

from cProfile import Profile
from pstats import SortKey, Stats

In [21]:
assert 'bw25_premise_background_v2' in bd.projects
if 'bw25_premise_background_v2' not in bd.projects:
    import bw2io as bi
    bi.backup.restore_project_directory(fp= 'your/path/to/file/brightway2-project-bw25_premise_background_v2-backup.26-March-2024-01-40PM.tar.gz',
                                    overwrite_existing=True)

In [22]:
bd.projects.set_current('bw25_premise_background_v2')

to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_medium']
Databases dictionary with 5 object(s):
	biosphere3
	cutoff39
	db_2020
	db_2030
	db_2040


In [79]:
def create_standardized_test_system(db_name, number_tiers, number_processes_per_tier, td_timesteps, td_dtype, background_process, loop = False):
    """
    Create a foreground database with a standardized structure for testing purposes. It contains number_tiers supply chain levels, with each containing 
    number_processes_per_tier processes. Each process consumes from all processes in the tier below. 
    Each process has a temporal distribution with td_timesteps steps, e.g. 3, in td_dtype resolution, e.g. years, starting at a unit of time later than the consuming process
    The last tier consumes a process from the background system, with the same temporal distribution as above.
    If loop = True, a loop is included: the last process in last tier consumes from the last process in first tier, no TDs at this node.
    No additional TDs at biosphere flows.

    Parameters:
    db_name: str, name of the database
    number_tiers: int, number of tiers in the supply chain
    number_processes_per_tier: int, number of processes per tier
    td_timesteps: int, number of timesteps in the temporal distribution
    td_dtype: str, dtype of the temporal distribution, e.g. "timedelta64[Y]" or "timedelta64[M]"
    background_process: node, bw2data object, background process to be consumed by the last tier in the foreground system
    loop: bool, whether to include a loop in the foreground system

    returns: None

    
    """
    
    # create a new database
    if db_name in bd.databases:
        del bd.databases[db_name]
        print(f"Database {db_name} already exists, overwriting.")
    foreground = bd.Database(db_name)
    foreground.register()

    # save FU node
    FU_node = foreground.new_node(
        code="FU",
        name="functional unit",
        unit = "unit"
    )
    FU_node.save()
    FU_node.new_edge(input=FU_node , amount=1, unit = "unit", type="production").save()

    #create nodes
    for tier in range(1, number_tiers+1):
        for process in range(1, number_processes_per_tier+1):
            process_name = f'tier{tier}_act{process}'
            process_code = f't{tier}_a{process}'
            process_node = foreground.new_node(
                code=process_code,
                name=process_name,
                unit = "unit"
            )
            process_node.save()
            process_node.new_edge(input=process_node , amount=1, unit = "unit", type="production").save()

    #relink nodes
    for tier in range(1, number_tiers+1):
        for process in range(1, number_processes_per_tier+1):
            process_name = f'tier{tier}_act{process}'
            process_code = f't{tier}_a{process}'
            process_node = foreground.get(process_code)
            if tier == 1: #tier 1 = FU
                consuming_node = FU_node
                new_edge = consuming_node.new_edge(input=process_node , amount=1, unit = "unit", type="technosphere")
                new_edge.save()
            else: # lower tiers: all nodes at the tier above consume from all nodes at the tier below, with a temporal distribution specified by the inputs
                for process in range(1, number_processes_per_tier+1):
                    consuming_node = foreground.get(f't{tier-1}_a{process}')
                    new_edge = consuming_node.new_edge(input=process_node , amount=1, unit = "unit", type="technosphere")
                    temporal_distribution = TemporalDistribution(date = np.array(np.arange(1, td_timesteps+1), dtype = td_dtype), 
                                                                amount = np.full(td_timesteps, 1 / td_timesteps)) #linear distribution between 1 and td_timesteps in years
                    new_edge["temporal_distribution"] = temporal_distribution
                    new_edge.save()

    # add background process to nodes on last tier
    for process in range(1, number_processes_per_tier+1):
        last_foreground_node = foreground.get(f't{number_tiers}_a{process}')
        new_edge = last_foreground_node.new_edge(input=background_process, amount=4, type="technosphere")
        temporal_distribution = TemporalDistribution(date = np.array(np.arange(1, td_timesteps+1), dtype = td_dtype),
                                                        amount = np.full(td_timesteps, 1 / td_timesteps)) #linear distribution between 1 and td_timesteps in years
        new_edge["temporal_distribution"] = temporal_distribution
        new_edge.save()

    if loop:
        # add loop to foreground: last process in last tier consumes from last process in first tier, no TDs
        loop_consumer = foreground.get(f't{number_tiers}_a{number_processes_per_tier}')
        loop_producer = foreground.get(f't{1}_a{number_processes_per_tier}')
        new_edge = loop_consumer.new_edge(input=loop_producer, amount=0.3, type="technosphere")
        new_edge.save()

    count_exc = 0
    for act in foreground:
        for exc in act.technosphere():
            count_exc += 1
            
    print(f"Database {db_name} created with {len(foreground)} processes and {count_exc} exchanges")

### 1) small system

In [ ]:
background_process = bd.get_node(database="db_2020", name="market for transport, passenger car, electric", location="GLO")
db_name = "test_small"
create_standardized_test_system(db_name = db_name, number_tiers = 2 , number_processes_per_tier=2,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", background_process=background_process,
                                loop = False)

Database test_small created with 5 processes and 8 exchanges


In [25]:
# # check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

In [26]:
database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [27]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_date_dict=database_date_dict,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2766164 function calls (2762178 primitive calls) in 5.789 seconds

   Ordered by: cumulative time
   List reduced from 995 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.736    5.736 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:91(__init__)
        1    0.000    0.000    2.816    2.816 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.000    0.000    2.425    2.425 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    2.356    2.356 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:366(solve_linear_system)
        1    0.000    0.000    2.356    2.356 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\si

In [28]:
with Profile() as profile:
    tlca.build_timeline()
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 10
         6242762 function calls (6017710 primitive calls) in 36.908 seconds

   Ordered by: cumulative time
   List reduced from 1473 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       18    0.441    0.025   29.908    1.662 {built-in method select.select}
    72339   29.076    0.000   29.093    0.000 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\helper_classes.py:109(add)
    72592    0.059    0.000    4.486    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4664(next)
    72572    0.086    0.000    4.427    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:4578(iterate)
    72441    0.200    0.000    3.339    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:7894(process_row)
    72441    0.231  

In [29]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         3799096 function calls (3714640 primitive calls) in 9.979 seconds

   Ordered by: cumulative time
   List reduced from 1414 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       74    0.002    0.000    4.709    0.064 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
  166/121    0.000    0.000    4.263    0.035 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
      121    0.000    0.000    4.261    0.035 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2087(execute)
      121    0.001    0.000    4.261    0.035 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2260(_execute)
      121    0.000    0.000    4.259    0.035 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packag

In [30]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2489 function calls (2481 primitive calls) in 0.005 seconds

   Ordered by: cumulative time
   List reduced from 248 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\mapped_matrix.py:15(__init__)
        1    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\mapped_matrix.py:167(map_indices)
        1    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\resource_group.py:166(map_indices)
        1    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\ipykernel\iostream.py:276(<lambda>)
        8    0.000    0.000    0.001    0.000 c:\Users\MULLERA\Ap

In [31]:
tlca.static_score

2.058617754929586

### 2) medium system

In [39]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_medium']
Databases dictionary with 5 object(s):
	biosphere3
	cutoff39
	db_2020
	db_2030
	db_2040


In [57]:
background_process = bd.get_node(database="db_2020", name="market for transport, passenger car, electric", location="GLO")
db_name = "test_medium"
create_standardized_test_system(db_name = db_name, number_tiers = 4, number_processes_per_tier=4,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", background_process=background_process,
                                loop = False)

Database test_medium created with 17 processes and 56 exchanges


In [58]:
# check db entries:
foreground = bd.Database(db_name)
for act in foreground:
    print(act)
    for exc in act.exchanges():
        td = exc.get("temporal_distribution", None)
        print("--", exc["type"],"|", exc,"|", td)  
    print ("\n")

'tier4_act1' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier4_act1' (unit, GLO, None) to 'tier4_act1' (unit, GLO, None)> | None
-- technosphere | Exchange: 4 kilometer 'market for transport, passenger car, electric' (kilometer, GLO, None) to 'tier4_act1' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1


'tier2_act4' (unit, GLO, None)
-- production | Exchange: 1 unit 'tier2_act4' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | None
-- technosphere | Exchange: 1 unit 'tier3_act1' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tier3_act2' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tier3_act3' (unit, GLO, None) to 'tier2_act4' (unit, GLO, None)> | TemporalDistribution instance with 2 values and total: 1
-- technosphere | Exchange: 1 unit 'tie

In [59]:
database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    db_name: "dynamic",
}

In [60]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_date_dict=database_date_dict,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2890863 function calls (2856774 primitive calls) in 6.837 seconds

   Ordered by: cumulative time
   List reduced from 1017 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.575    3.575 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.000    0.000    3.141    3.141 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    2.223    2.223 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:366(solve_linear_system)
        1    0.000    0.000    2.223    2.223 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\pypardiso\scipy_aliases.py:12(spsolve)
        2    2.218    1.109    2.218    1.109 c:\Users\MULLERA\AppData\Local\anaconda3\envs\t

In [61]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10) #1 min

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 596
         10787689 function calls (10462928 primitive calls) in 47.443 seconds

   Ordered by: cumulative time
   List reduced from 1475 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    0.046    0.003   38.013    2.376 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:313(_select)
    72561   31.020    0.000   31.037    0.000 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\helper_classes.py:109(add)
       16    0.427    0.027   29.149    1.822 {built-in method select.select}
        1    0.001    0.001    9.666    9.666 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:151(build_timeline)
    72318    0.310    0.000    8.752    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\backends\base.py:316(__iter__)
2942/2940    0.003  

In [62]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10) #37 secs

         12974572 function calls (12803202 primitive calls) in 83.762 seconds

   Ordered by: cumulative time
   List reduced from 1424 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      766    0.021    0.000   70.982    0.093 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
2008/1405    0.003    0.000   69.643    0.050 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner)
     1405    0.001    0.000   69.628    0.050 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2087(execute)
     1202    0.002    0.000   69.504    0.058 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:7254(__iter__)
     1405    0.008    0.000   68.888    0.049 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-pac

In [53]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2852 function calls (2850 primitive calls) in 0.006 seconds

   Ordered by: cumulative time
   List reduced from 229 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.005    0.005 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\asyncio\base_events.py:1910(_run_once)
        1    0.000    0.000    0.005    0.005 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:319(select)
        1    0.000    0.000    0.005    0.005 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:313(_select)
        1    0.000    0.000    0.005    0.005 {built-in method select.select}
        1    0.000    0.000    0.005    0.005 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:360(static_lcia)
        1    0.000    0.000    0.005    0.005 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex

In [56]:
tlca.static_score

11.95350274003336

### 3) large system


In [63]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_medium']
Databases dictionary with 5 object(s):
	biosphere3
	cutoff39
	db_2020
	db_2030
	db_2040


In [70]:
background_process = bd.get_node(database="db_2020", name="market for transport, passenger car, electric", location="GLO")
db_name = "test_large"
create_standardized_test_system(db_name = db_name, number_tiers = 5, number_processes_per_tier=4,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", background_process=background_process,
                                loop = False)

Database test_large already exists, overwriting.
Database test_large created with 21 processes and 72 exchanges


In [71]:
# # check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

In [72]:
database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    db_name: "dynamic"}

In [73]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_date_dict=database_date_dict,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2932530 function calls (2896291 primitive calls) in 6.285 seconds

   Ordered by: cumulative time
   List reduced from 1043 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.683    2.683 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.000    0.000    2.253    2.253 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    2.210    2.210 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:366(solve_linear_system)
        1    0.000    0.000    2.210    2.210 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\pypardiso\scipy_aliases.py:12(spsolve)
        2    2.203    1.101    2.203    1.101 c:\Users\MULLERA\AppData\Local\anaconda3\envs\t

In [74]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal


c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_graph_tools\graph_traversal.py:473: UserWarning: Stopping traversal due to calculation count.
  warnings.warn("Stopping traversal due to calculation count.")


Calculation count: 2001
         19319170 function calls (18681003 primitive calls) in 56.084 seconds

   Ordered by: cumulative time
   List reduced from 1477 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.039    0.002   43.982    2.199 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:313(_select)
       20    0.462    0.023   37.743    1.887 {built-in method select.select}
    72701   31.270    0.000   31.287    0.000 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\helper_classes.py:109(add)
        1    0.001    0.001   11.424   11.424 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:151(build_timeline)
        1    0.001    0.001    9.706    9.706 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:29(__init__)
        1    0.070    0.070    9.705    9.705 C:\Users\MULLERA\O

In [75]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         17210206 function calls (16980999 primitive calls) in 124.671 seconds

   Ordered by: cumulative time
   List reduced from 1378 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  124.670  124.670 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:268(lci)
       26    0.001    0.000  112.243    4.317 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\asyncio\base_events.py:1910(_run_once)
       26    0.606    0.023  112.216    4.316 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:319(select)
     1219    0.034    0.000  109.487    0.090 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
3249/2262    0.004    0.000  107.953    0.048 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner

In [76]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         5358 function calls (5354 primitive calls) in 0.008 seconds

   Ordered by: cumulative time
   List reduced from 241 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       25    0.002    0.000    0.003    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_processing\datapackage.py:180(filter_by_attribute)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\mapped_matrix.py:15(__init__)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\utils.py:19(consistent_global_index)
        1    0.000    0.000    0.001    0.001 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\matrix_utils\utils.py:6(filter_groups_for_packages)
        2    0.000    0.000    0.001    0.001

In [77]:
tlca.static_score

379.79756051343105

### 4) small system with loop

In [78]:
# needed to avoid multiple results error https://github.com/brightway-lca/bw_timex/issues/101
to_delete = []
for db in bd.databases: #deleting all test_systems
    if "test" in db:
        to_delete.append(db)
print(to_delete)

for db in to_delete:
    del bd.databases[db] 

print(bd.databases)

['test_large']
Databases dictionary with 5 object(s):
	biosphere3
	cutoff39
	db_2020
	db_2030
	db_2040


In [80]:
background_process = bd.get_node(database="db_2020", name="market for transport, passenger car, electric", location="GLO")
db_name = "test_loop"
create_standardized_test_system(db_name = db_name, number_tiers = 2, number_processes_per_tier=3,
                                td_timesteps= 2, td_dtype = "timedelta64[Y]", background_process=background_process,
                                loop = True)

Database test_loop created with 7 processes and 16 exchanges


In [ ]:
# # check db entries:
# foreground = bd.Database(db_name)
# for act in foreground:
#     print(act)
#     for exc in act.exchanges():
#         td = exc.get("temporal_distribution", None)
#         print("--", exc["type"],"|", exc,"|", td)  
#     print ("\n")

In [81]:
database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    db_name: "dynamic"}

In [82]:
from bw_timex import TimexLCA
FU_node = bd.Database(db_name).get("FU")

with Profile() as profile:
    tlca = TimexLCA(
        demand={FU_node: 1},
        method=("EF v3.1", "climate change", "global warming potential (GWP100)"),
        database_date_dict=database_date_dict,
    )
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         2787235 function calls (2758095 primitive calls) in 5.767 seconds

   Ordered by: cumulative time
   List reduced from 1024 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.949    2.949 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:384(lci)
        1    0.000    0.000    2.543    2.543 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:411(lci_calculation)
        1    0.000    0.000    2.492    2.492 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:366(solve_linear_system)
        1    0.000    0.000    2.492    2.492 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\pypardiso\scipy_aliases.py:12(spsolve)
        2    2.485    1.242    2.485    1.243 c:\Users\MULLERA\AppData\Local\anaconda3\envs\t

In [83]:
with Profile() as profile:
    tlca.build_timeline()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:208: UserWarning: No edge filter function provided. Skipping all edges in background databases.
  warnings.warn(


Starting graph traversal
Calculation count: 339


C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2041-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2042-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2043-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:537: Warning: Reference date 2044-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(
C:\Users\MULLERA\OneDrive - VITO

         10579214 function calls (10252086 primitive calls) in 54.938 seconds

   Ordered by: cumulative time
   List reduced from 1475 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       21    1.269    0.060   38.676    1.842 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:313(_select)
    72748   34.009    0.000   34.028    0.000 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\helper_classes.py:109(add)
       21    0.414    0.020   20.773    0.989 {built-in method select.select}
        1    0.002    0.002   17.646   17.646 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:151(build_timeline)
        1    0.009    0.009    7.540    7.540 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timeline_builder.py:108(build_timeline)
    72318    0.279    0.000    6.642    0.000 c:\Users\MULLERA\AppData\Local\anac

In [84]:
with Profile() as profile:
    tlca.lci()
    
Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         62205535 function calls (61973453 primitive calls) in 157.458 seconds

   Ordered by: cumulative time
   List reduced from 1378 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  157.449  157.449 C:\Users\MULLERA\OneDrive - VITO\Documents\04_Coding\tictac_lca\bw_timex\timex_lca.py:268(lci)
       32    0.001    0.000  112.742    3.523 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\asyncio\base_events.py:1910(_run_once)
       32    3.478    0.109  112.536    3.517 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\selectors.py:319(select)
     1452    0.050    0.000  104.599    0.072 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2data\utils.py:362(get_node)
3007/2293    0.005    0.000   96.948    0.042 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\peewee.py:2012(inner

In [85]:
with Profile() as profile:
    tlca.static_lcia()

Stats(profile).sort_stats(SortKey.CUMULATIVE).print_stats(10)

         7571 function calls (7567 primitive calls) in 0.008 seconds

   Ordered by: cumulative time
   List reduced from 241 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       25    0.002    0.000    0.003    0.000 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw_processing\datapackage.py:180(filter_by_attribute)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\utils.py:19(consistent_global_index)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\bw2calc\lca.py:446(lcia_calculation)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppData\Local\anaconda3\envs\timex_temporalis_local\Lib\site-packages\scipy\sparse\_matrix.py:43(__mul__)
        1    0.000    0.000    0.002    0.002 c:\Users\MULLERA\AppD

In [86]:
tlca.static_score

6.2748978774385025